# py-iku: Basic Usage Guide

This notebook demonstrates the core functionality of py-iku - converting Python data processing code to Dataiku DSS flows.

## What is py-iku?

py-iku is a Python library that:
- Converts pandas, NumPy, and scikit-learn code to Dataiku DSS recipes
- Generates visual flow diagrams
- Exports directly to Dataiku DSS projects

## Installation

```bash
pip install py2dataiku
```

In [1]:
# !pip install -e ..

In [2]:
# Import the library
from py2dataiku import convert, Py2Dataiku

## 1. Simple Conversion

Let's start with a simple pandas pipeline that reads data, cleans it, and saves the result.

In [3]:
# Define some pandas code
pandas_code = '''
import pandas as pd

# Load raw data
df = pd.read_csv('customers.csv')

# Clean the data
df['name'] = df['name'].str.strip().str.title()
df['email'] = df['email'].str.lower()
df = df.dropna(subset=['customer_id'])

# Save cleaned data
df.to_csv('customers_cleaned.csv', index=False)
'''

# Convert to Dataiku flow
flow = convert(pandas_code)

print(f"Flow: {flow.name}")
print(f"Datasets: {len(flow.datasets)}")
print(f"Recipes: {len(flow.recipes)}")

Flow: converted_flow
Datasets: 2
Recipes: 1


In [4]:
# Get a text summary of the flow
print(flow.get_summary())

Flow: converted_flow
Source: unknown
Generated: 2026-01-13T20:30:48.239125

Datasets: 2
  - Input: 1
  - Intermediate: 0
  - Output: 1

Recipes: 1
  - prepare: 1


## 2. Visualizing Flows

py-iku supports multiple visualization formats.

In [5]:
# ASCII visualization (great for terminals)
print(flow.visualize(format='ascii'))

════════════════════════════════════════════════════════════════════════════════
                          DATAIKU FLOW: converted_flow  
════════════════════════════════════════════════════════════════════════════════

                              ┌──────────────────┐
                              │ 📊 df             │
                             │    [INPUT]        │
                              └──────────────────┘

                                       │
                                       ▼

                                ┌──────────────┐
                               │   ⚙ PREPARE   │
                                │ ──────────── │
                                └──────────────┘

                                       │
                                       ▼

                              ┌──────────────────┐
                              │ 📊 df_prepared    │
                             │    [OUTPUT]       │
                              └──────────────────┘

═══════

In [6]:
# Mermaid diagram (for documentation)
print(flow.visualize(format='mermaid'))

flowchart TD
    subgraph inputs[Input Datasets]
        D0[(df)]
    end
    subgraph outputs[Output Datasets]
        D1[(df_prepared)]
    end
    R0{Prepare\n(1 steps)}
    D0 --> R0
    R0 --> D1

    style D0 fill:#e1f5fe
    style D1 fill:#c8e6c9
    style R0 fill:#fff3e0


In [7]:
# HTML visualization (interactive)
from IPython.display import HTML
HTML(flow.visualize(format='html'))

## 3. Complex Pipeline Example

Let's convert a more complex pipeline with joins, grouping, and filtering.

In [8]:
complex_code = '''
import pandas as pd

# Load multiple data sources
orders = pd.read_csv('orders.csv')
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')

# Join orders with customers
orders_enriched = pd.merge(orders, customers, on='customer_id', how='left')

# Join with products
orders_full = pd.merge(orders_enriched, products, on='product_id', how='left')

# Filter to high-value orders
high_value = orders_full[orders_full['amount'] >= 100]

# Aggregate by customer
customer_summary = high_value.groupby('customer_id').agg({
    'amount': ['sum', 'mean', 'count'],
    'order_date': 'max'
}).reset_index()

# Save results
customer_summary.to_csv('customer_summary.csv', index=False)
'''

flow = convert(complex_code)
print(flow.get_summary())

Flow: converted_flow
Source: unknown
Generated: 2026-01-13T20:30:48.279578

Datasets: 7
  - Input: 3
  - Intermediate: 4
  - Output: 0

Recipes: 4
  - grouping: 1
  - join: 2
  - split: 1


In [9]:
# View the flow diagram
print(flow.visualize(format='ascii'))

════════════════════════════════════════════════════════════════════════════════
                          DATAIKU FLOW: converted_flow  
════════════════════════════════════════════════════════════════════════════════

                              ┌──────────────────┐
                              │ 📊 orders         │
                             │    [INPUT]        │
                              └──────────────────┘

                              ┌──────────────────┐
                              │ 📊 customers      │
                             │    [INPUT]        │
                              └──────────────────┘

                              ┌──────────────────┐
                              │ 📊 products       │
                             │    [INPUT]        │
                              └──────────────────┘

                                       │
                                       ▼

                                ┌──────────────┐
                               │ 

## 4. Examining Recipes

Let's look at the individual recipes generated.

In [10]:
# List all recipes
for recipe in flow.recipes:
    print(f"\nRecipe: {recipe.name}")
    print(f"  Type: {recipe.recipe_type.value}")
    print(f"  Inputs: {[d.name for d in recipe.inputs]}")
    print(f"  Outputs: {[d.name for d in recipe.outputs]}")


Recipe: join_1
  Type: join


AttributeError: 'str' object has no attribute 'name'

## 5. Export Formats

Export the flow in different formats.

In [11]:
# Export as JSON
import json
print(json.dumps(flow.to_dict(), indent=2)[:1000] + '...')

{
  "flow_name": "converted_flow",
  "generated_from": null,
  "generation_timestamp": "2026-01-13T20:30:48.279578",
  "total_recipes": 4,
  "total_datasets": 7,
  "datasets": [
    {
      "name": "orders",
      "type": "input",
      "schema": [],
      "source_variable": "orders",
      "source_line": 5,
      "notes": []
    },
    {
      "name": "customers",
      "type": "input",
      "schema": [],
      "source_variable": "customers",
      "source_line": 6,
      "notes": []
    },
    {
      "name": "products",
      "type": "input",
      "schema": [],
      "source_variable": "products",
      "source_line": 7,
      "notes": []
    },
    {
      "name": "orders_enriched",
      "type": "intermediate",
      "schema": [],
      "source_variable": null,
      "source_line": null,
      "notes": []
    },
    {
      "name": "orders_full",
      "type": "intermediate",
      "schema": [],
      "source_variable": null,
      "source_line": null,
      "notes": []
    },
 

In [12]:
# Export as YAML
print(flow.to_yaml()[:1000] + '...')

flow_name: converted_flow
generated_from: null
generation_timestamp: '2026-01-13T20:30:48.279578'
total_recipes: 4
total_datasets: 7
datasets:
- name: orders
  type: input
  schema: []
  source_variable: orders
  source_line: 5
  notes: []
- name: customers
  type: input
  schema: []
  source_variable: customers
  source_line: 6
  notes: []
- name: products
  type: input
  schema: []
  source_variable: products
  source_line: 7
  notes: []
- name: orders_enriched
  type: intermediate
  schema: []
  source_variable: null
  source_line: null
  notes: []
- name: orders_full
  type: intermediate
  schema: []
  source_variable: null
  source_line: null
  notes: []
- name: high_value
  type: intermediate
  schema: []
  source_variable: null
  source_line: null
  notes: []
- name: _chain_step_0
  type: intermediate
  schema: []
  source_variable: null
  source_line: null
  notes: []
recipes:
- name: join_1
  type: join
  inputs:
  - orders
  - customers
  outputs:
  - orders_enriched
  source

## 6. Using the Py2Dataiku Class

For more control, use the `Py2Dataiku` class directly.

In [13]:
# Create converter instance
converter = Py2Dataiku()

# Convert code
flow = converter.convert(pandas_code)

# Validate the flow
is_valid, errors = flow.validate()
print(f"Flow valid: {is_valid}")
if errors:
    print(f"Errors: {errors}")

ValueError: too many values to unpack (expected 2)

## Next Steps

- See `02_numpy_operations.ipynb` for NumPy support
- See `03_sklearn_pipelines.ipynb` for scikit-learn support
- See `04_visualizations.ipynb` for visualization options
- See `05_advanced_features.ipynb` for plugins and DSS export